In [2]:
import json
data = {}
data["truck_types"] = [
    {
        'type': 'BigTruck',
        'speed': 45,
        'fuel_consumption': 0.25,
        'fuel_capacity': 300,
        'capacities': {'mixed': 35000},
        'unloading_time': 0.05
    }
]
data["container_types"] = [
    {
        'type': 'RegularContainer',
        'capacity': 1100,
        'garbage_type': 'mixed',
        'processing_time': 0.05
    }
]

data["garbage_types"] = ["mixed"]


with open("lviv_config.json", "w") as f:
    f.write(json.dumps(data, indent=4))

In [3]:
def create_input(original_input, output, trucks_count):

    with open(original_input) as f:
        data = json.load(f)
    for cont in data["containers"]:
        cont["type"] = "RegularContainer"
    truck = data["trucks"][0]
    truck["type"] = "BigTruck"
    data["trucks"] = [truck] * trucks_count
    data["landfills"][0]["latitude"] = 49.9012481;
    data["landfills"][0]["longitude"] = 24.0300928;
    with open(output, "w") as f:
        f.write(json.dumps(data, indent=4))


In [6]:
import subprocess
import sys
import os
import time


PROGRAM_PATH = "./routing-problem"


def create_dir(path):
    if os.path.exists(path):
        return

    os.makedirs(path)


def run(params):
    args = [f"--{name}={value}" for name, value in params.items()]
    command = f"{PROGRAM_PATH} {' '.join(args)}"
    res = os.system(command)
    if res != 0:
        print("ERROR")
        print("Command:", command)
        print("RES:", res)
        
def generate_params(search_data):
    keys = []
    all_params = [] 
    for key in search_data:
        if type(search_data[key]) == list:
            all_params.append(search_data[key])
        else:
            all_params.append([search_data[key]])
        keys.append(key)
    finished = False
    
    indices = [0] * len(keys)
    while not finished:
        params = {keys[i]: all_params[i][indices[i]] for i in range(len(keys))}
        yield params
        for i in range(len(keys)):
            index = len(keys) - i - 1
            if len(all_params[index]) - 1 == indices[index]:
                indices[index] = 0
                if index == 0:
                    finished = True
                    break
            else:
                indices[index] += 1
                break
                    
        
def perform_becnhmarks(searches_data, base_folder, filename, config, original_file="data/Lviv/input.json"):
    create_dir(base_folder)
    i = 0
    for search_data in searches_data:
        for params in generate_params(search_data):
            for key, value in params.items():
                if key == "output" or key == "input":
                    continue
                print(f"{key}={value},", end=" ")
                #if key == "trucks_count":           
                    #create_input(original_file, filename, value)
            if i <= 3:
                i += 1
                continue
            print(f"\nIteration: {i}")

            del params["trucks_count"]
            params["config"]  = config
            params["output"] = f"{base_folder}/output_{i}.json"
            params["input"] = filename
            params["real"] = True
            start_time = time.time()
            run(params)
            execution_time = time.time() - start_time
            with open(params["output"]) as f:
                output = json.load(f)
            for param in params:
                output[param] = params[param]
            output["execution_time"] = execution_time
            print("Execution time", execution_time)
            json_string = json.dumps(output, indent=4)
            with open(params["output"], "w") as f:
                f.write(json_string)
            i += 1

In [ ]:
heuristics = [
    "MOVE_IN_EXPORTATION",
    "REMOVE_LOOPS_SWAP_ANY_EXPORTATION",
    "REMOVE_LOOPS_MOVE_EXPORTATION",
    "ANY_EXPORTATION_PROB1",
    "ANY_MOVE_EXPORTATION_FIXED2",
    "ANY_MOVE_EXPORTATION",
    "ANY_EXPORTATION_TIME1",
    "ANY_EXPORTATION",
    "ANY_MOVE_SWAP",
]

swap_any_heuristics = [
    "ANY_MOVE_SWAP",
    "ANY_EXPORTATION_PROB1",
    "REMOVE_LOOPS_SWAP_ANY_EXPORTATION",
    "ANY_MOVE_EXPORTATION_FIXED2",
    "ANY_EXPORTATION_TIME1",
    "ANY_EXPORTATION",
    "ANY_MOVE_EXPORTATION",
]

searches = [
    {
        "search": "NO_SEARCH",
        "base_solution": ["CLUSTER", "SIMPLE_GREEDY"],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "TABU",
        "heuristic": heuristics,
        "base_solution": ["SIMPLE_GREEDY"],
        "tabu_queue_coef": [0.3, 0.5],
        "iterations": [200, 500],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "ANNEALING",
        "heuristic": heuristics[:3],
        "base_solution":  ["SIMPLE_GREEDY"],
        "temp_coef": [0.00001, 0.0001],
        "iterations": [3000, 5000],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "TABU",
        "heuristic": swap_any_heuristics,
        "base_solution": ["SIMPLE_GREEDY"],
        "tabu_queue_coef": [0.3],
        "iterations": [500],
        "trucks_count": [10, 15, 20],
        "swap_any_to_generate": [500]
    },
    {
        "search": "ANNEALING",
        "heuristic": heuristics[3:],
        "base_solution":  ["SIMPLE_GREEDY"],
        "temp_coef": [0.00001, 0.0001],
        "iterations": [3000, 5000],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "GENETIC",
        "heuristic": heuristics,
        "base_solution": ["SIMPLE_GREEDY"],
        "mutation_probability": [0.1, 0.3],
        "iterations": [200, 400],
        "population_size": [10, 20],
        "trucks_count": [10, 15, 20]
    },
        {
        "search": "TABU",
        "heuristic": heuristics,
        "base_solution": ["CLUSTER"],
        "tabu_queue_coef": [0.3, 0.5],
        "iterations": [200, 500],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "ANNEALING",
        "heuristic": heuristics,
        "base_solution": ["CLUSTER"],
        "temp_coef": [0.00001, 0.0001],
        "iterations": [3000, 5000],
        "trucks_count": [10, 15, 20]
    },
    {
        "search": "GENETIC",
        "heuristic": heuristics,
        "base_solution": ["CLUSTER"],
        "mutation_probability": [0.1, 0.3],
        "iterations": [200, 400],
        "population_size": [10, 20],
        "trucks_count": [10, 15, 20]
    },
    
]


perform_becnhmarks(searches, "results_lviv", "lviv_input.json", "lviv_config.json")

search=NO_SEARCH, base_solution=CLUSTER, trucks_count=10, 
Iteration: 0


In [11]:
current_count = 10
map_file = "data/SantBoiDeLlobregat/osrm/spain-latest.osrm"
best_searches = [
    {
        "search": "TABU",
        "heuristic": "ANY_MOVE_EXPORTATION",
        "base_solution": "SIMPLE_GREEDY",
        "tabu_queue_coef": 0.3,
        "iterations": 500,
        "trucks_count": current_count,
        "map_file": map_file
        
    },
    {
        "search": "TABU",
        "heuristic": "ANY_MOVE_EXPORTATION",
        "base_solution": "SIMPLE_GREEDY",
        "tabu_queue_coef": 0.3,
        "iterations": 500,
        "trucks_count": current_count,
        "swap_any_to_generate": 500,
        "map_file": map_file
    },
    {
        "search": "ANNEALING",
        "heuristic": "ANY_MOVE_EXPORTATION",
        "base_solution":  "SIMPLE_GREEDY",
        "temp_coef": 0.00001,
        "iterations": 5000,
        "trucks_count": current_count,
        "map_file": map_file
    },
    {
        "search": "GENETIC",
        "heuristic": "ANY_MOVE_EXPORTATION_FIXED2",
        "base_solution": "SIMPLE_GREEDY",
        "mutation_probability": 0.1,
        "iterations": 500,
        "population_size": 10,
        "trucks_count": current_count,
        "map_file": map_file
    },
    {
        "search": "NO_SEARCH",
        "base_solution": "SIMPLE_GREEDY",
        "trucks_count": current_count,
        "map_file": map_file
    }
    
]

perform_becnhmarks(best_searches,
                   "data/SantBoiDeLlobregat/best_results",
                   "data/SantBoiDeLlobregat/input.json",
                   "lviv_config.json")

search=TABU, heuristic=ANY_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=500, trucks_count=10, map_file=data/SantBoiDeLlobregat/osrm/spain-latest.osrm, search=TABU, heuristic=ANY_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, tabu_queue_coef=0.3, iterations=500, trucks_count=10, swap_any_to_generate=500, map_file=data/SantBoiDeLlobregat/osrm/spain-latest.osrm, search=ANNEALING, heuristic=ANY_MOVE_EXPORTATION, base_solution=SIMPLE_GREEDY, temp_coef=1e-05, iterations=5000, trucks_count=10, map_file=data/SantBoiDeLlobregat/osrm/spain-latest.osrm, search=GENETIC, heuristic=ANY_MOVE_EXPORTATION_FIXED2, base_solution=SIMPLE_GREEDY, mutation_probability=0.1, iterations=500, population_size=10, trucks_count=10, map_file=data/SantBoiDeLlobregat/osrm/spain-latest.osrm, search=NO_SEARCH, base_solution=SIMPLE_GREEDY, trucks_count=10, map_file=data/SantBoiDeLlobregat/osrm/spain-latest.osrm, 
Iteration: 4
2276.58
2276.58
Code: NoRoute
Message: NoRoute
Execution time 